In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Novel Coronavirus 2019 dataset

In [3]:
df_confirmed_raw = pd.read_csv("Data/00 Core/Novel Coronavirus 2019 datasets/time_series_covid_19_confirmed.csv")
df_deaths_raw = pd.read_csv("Data/00 Core/Novel Coronavirus 2019 datasets/time_series_covid_19_deaths.csv")
df_recovered_raw = pd.read_csv("Data/00 Core/Novel Coronavirus 2019 datasets/time_series_covid_19_recovered.csv")

In [4]:
df_confirmed = (pd.melt(df_confirmed_raw, id_vars="Country/Region", value_vars=df_confirmed_raw.columns[4:])
          .rename(columns={"variable": "Date", "value": "confirmed"})
          .assign(Date=lambda x: pd.to_datetime(x.Date))
          .groupby(["Country/Region", "Date"])
          .sum()
          .reset_index().rename(columns={"Country/Region": "Country"}))

df_deaths = (pd.melt(df_deaths_raw, id_vars="Country/Region",value_vars=df_deaths_raw.columns[4:])
          .rename(columns={"variable": "Date", "value": "deaths"})
          .assign(Date=lambda x: pd.to_datetime(x.Date))
          .groupby(["Country/Region", "Date"])
          .sum()
          .reset_index().rename(columns={"Country/Region": "Country"}))

df_recovered = (pd.melt(df_recovered_raw, id_vars="Country/Region",value_vars=df_recovered_raw.columns[4:])
          .rename(columns={"variable": "Date", "value": "recovered"})
          .assign(Date=lambda x: pd.to_datetime(x.Date))
          .groupby(["Country/Region", "Date"])
          .sum()
          .reset_index().rename(columns={"Country/Region": "Country"}))


In [5]:
df_merged = pd.merge(df_confirmed, df_deaths, how="outer", on=["Country", "Date"])
df_merged = pd.merge(df_merged, df_recovered, how="outer", on=["Country", "Date"])

In [8]:
pd.to_pickle(df_merged, "Data/novel_clean.pkl")
df_merged.to_csv("Data/Novel Coronavirus Clean.csv")

### Governance tracker

In [9]:
df_CGRT_raw = pd.read_csv("Data/00 Core/Oxford Covid-19 Government Response Tracker/OxCGRT_latest.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.



In [10]:
display_index_cols = [col for col in df_CGRT_raw.columns if 'index' in col.lower() and 'display' in col.lower()]

df_CGRT = (df_CGRT_raw
           .assign(Date=lambda x: pd.to_datetime(x.Date, format="%Y%m%d"))
           .groupby(["CountryName", "CountryCode", "Date"])
           [display_index_cols]
           .mean()
           .reset_index())

In [13]:
pd.to_pickle(df_CGRT, "Data/governance_clean.pkl")
df_CGRT.to_csv("Data/Governance Response Tracker clean.csv")

### Community mobility

In [14]:
df_mobility_raw = pd.read_csv("Data/00 Core/Community mobility reports/Global_Mobility_Report.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



In [15]:
value_cols = list(df_mobility_raw.columns[8:])
df_mobility = df_mobility_raw.groupby(["country_region", "date"])[value_cols].mean().reset_index()

In [18]:
pd.to_pickle(df_mobility, "Data/mobility_clean.pkl")
df_mobility.to_csv("Data/Global Mobility Report clean.csv")

### Health systems data

In [19]:
df_health = pd.read_csv("Data/01 Suggested/World development indicators - health systems/2.12_Health_systems.csv")